In [4]:
# import the necessary packages
from tensorflow.keras.models import load_model
from skimage import transform
from skimage import exposure
from skimage import io
from imutils import paths
import numpy as np
import argparse
import imutils
import random
import cv2
import os
from tkinter import *
from tkinter import ttk

print("[INFO] loading model...")
model = load_model("output/trafficsignnet.model")
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]
print("[INFO] predicting...")
imagePaths = list(paths.list_images("gtsrb-german-traffic-sign/Test"))
random.shuffle(imagePaths)
imagePaths = imagePaths[:25]
imagePaths = ['gtsrb-german-traffic-sign/Test\\00067.png']

label = "null"

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # load the image, resize it to 32x32 pixels, and then apply
    # Contrast Limited Adaptive Histogram Equalization (CLAHE),
    # just like we did during training
    image = io.imread(imagePath)
    image = transform.resize(image, (32, 32))
    image = exposure.equalize_adapthist(image, clip_limit=0.1)

    # preprocess the image by scaling it to the range [0, 1]
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)

    # make predictions using the traffic sign recognizer CNN
    preds = model.predict(image)
    j = preds.argmax(axis=1)[0]
    label = labelNames[j]
    print(label)
    # load the image using OpenCV, resize it, and draw the label
    # on it
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=128)
    while(True):
        cv2.putText(image, label, (5, 15), cv2.FONT_HERSHEY_SIMPLEX,
                0.45, (0, 255, 0), 2)

        cv2.imshow("image", image)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
    p = os.path.sep.join(["examples", "{}.png".format(i)])
    cv2.imwrite(p, image)

cv2.destroyAllWindows()

#variable that stores current speed value
speed = 0

#variable that stores speed limit value
speed_lim = int(label[13:15])
print(speed_lim)

speed_json = {"speed_limit" : speed_lim}
with open('person.txt', 'w') as json_file:
  json.dump(person_dict, json_file)


#creating the main window
root = Tk()

#changing background color to grey
root.configure(background='grey')


speedLimLabel = Label(root, text = "Current Speed limit = " + str(speed_lim) + " Km/hr", fg = "white", bg = "grey", padx = 20, pady = 40)
speedLimLabel.config(font=("Courier", 44))
speedLimLabel.grid(row = 0, sticky = 'w')

ttk.Separator(root,orient=HORIZONTAL).grid(row=1, columnspan=5, sticky = 'ew')

currentSpeedLabel = Label(root, text = str(speed) + " Km/hr", fg = "white", bg = "grey", padx = 20, pady = 40)
currentSpeedLabel.config(font=("Courier", 44))
currentSpeedLabel.grid(row = 2, sticky = 'w')

#increaments speed by 5 on left click and checks if speed is breached
def leftClick(event):
    global speed
    speed = speed + 5
    currentSpeedLabel['text'] = str(speed) + " Km/hr"
    if speed > speed_lim:
        warningLabel['bg'] = "red"
        warningLabel['text'] = "Speed Limit breached"
    else:
        warningLabel['bg'] = "white"
        warningLabel['text'] = "Under Speed Limit"
    
#decreaments speed by 5 on left click and checks if speed is under normal
def rightClick(event):    
    global speed
    if speed > 0:
        speed = speed - 5
    else:
        speed = speed
    currentSpeedLabel['text'] = str(speed) + " Km/hr"
    if speed > speed_lim:
        warningLabel['bg'] = "red"
        warningLabel['text'] = "Speed Limit breached"
    else:
        warningLabel['bg'] = "white"
        warningLabel['text'] = "Under Speed Limit"


ttk.Separator(root,orient=HORIZONTAL).grid(row=3, columnspan=5, sticky = 'ew')


warningLabel = Label(root, text = "Under Speed Limit", fg = "black", bg = "white", padx = 20, pady = 40)
warningLabel.config(font=("Courier", 44))
warningLabel.grid(row = 4)

#calls left click function when anywhere left clicked on the window
root.bind('<Button-1>', leftClick)

#calls right click function when anywhere left clicked on the window
root.bind('<Button-3>', rightClick)

#runs the window until closed
root.mainloop()

[INFO] loading model...
[INFO] predicting...
Speed limit (50km/h)
50
